In [53]:
import pandas as pd
# Leer los archivos CSV asegurándonos de que los separadores sean correctos
df1 = pd.read_csv('../data/english/test.txt', sep=';', header=None, names=['text', 'emotion'], engine='python')
df2 = pd.read_csv('../data/english/train.txt', sep=';', header=None, names=['text', 'emotion'], engine='python')
df3 = pd.read_csv('../data/english/val.txt', sep=';', header=None, names=['text', 'emotion'], engine='python')

# Concatenar los tres DataFrames
df = pd.concat([df1, df2, df3], ignore_index=True)

In [54]:
# Mostrar solo las filas duplicadas
filas_repetidas = df[df.duplicated()]
filas_repetidas

,text,emotion
15846,i feel more adventurous willing to take risks ...,joy


In [55]:
# Mostrar solo las filas duplicadas
filas_repetidas = df[df.duplicated()]
filas_repetidas

,text,emotion
15846,i feel more adventurous willing to take risks ...,joy


In [56]:
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)

In [57]:
from huggingface_hub import snapshot_download, hf_hub_download
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import fasttext

# Ruta base para guardar modelos
base_path = r"C:\Users\leo23\OneDrive\Documentos\Fine_tuning_llama2"

# # Descargar FastText a directorio local
# local_fasttext_path = hf_hub_download(
#     repo_id="facebook/fasttext-language-identification",
#     filename="model.bin",
#     cache_dir=f"{base_path}/models/fasttext"  # Subdirectorio específico
# )

# # Cargar modelo FastText
# language_detector = fasttext.load_model(local_fasttext_path)

In [61]:
from collections import Counter

# Unir todos los textos en uno solo
all_text = ' '.join(df['text'].astype(str))

# Contar cada caracter
char_counts = Counter(all_text)

# Mostrar los caracteres y sus conteos ordenados por frecuencia
for char, count in char_counts.most_common():
    print(f"'{char}': {count}")

' ': 384962
'e': 211438
't': 138761
'i': 131571
'a': 122400
'o': 110336
'n': 104662
's': 87551
'l': 86335
'r': 72174
'h': 71353
'd': 59648
'f': 49754
'm': 45717
'u': 44343
'g': 40315
'y': 36107
'c': 32844
'w': 30907
'p': 25764
'b': 25597
'v': 16841
'k': 16448
'I': 4543
'j': 3316
'x': 2289
'z': 1118
'q': 938
'’': 472


In [59]:
# !pip install sentencepiece

In [60]:
import re

def clean_text(text):
    text = re.sub(r'  ', ' ', text)
    text = re.sub(r' span style .+', '', text)
    text = re.sub(r' style color .+', '', text)
    text = re.sub(r' width .+', '', text)  # Eliminar todo después de
    text = re.sub(r' rel bookmark .+', '', text)  # Eliminar todo después de
    text = re.sub(r' utf .+', '', text)  # Eliminar todo después de
    text = re.sub(r' br .+', '', text)
    text = re.sub(r' div .+', '', text)
    text = re.sub(r' a href', '', text)
    text = re.sub(r' href', '', text)
    text = re.sub(r' lh', '', text)
    text = re.sub(r' font', '', text)
    text = re.sub(r' https', '', text)
    text = re.sub(r' img ', '', text)
    text = re.sub(r'src', '', text)
    text = re.sub(r'http', '', text)
    text = re.sub(r' www', '', text)
    text = re.sub(r'www', '', text)
    text = re.sub(r' dir', '', text)
    text = re.sub(r' url ', '', text)
    text = re.sub(r' n ', '', text)
    text = re.sub(r' fn', '', text)
    text = re.sub(r' ltr', '', text)
    text = re.sub(r'(?i)\bim\b', 'I am', text)
    text = re.sub(r'(?i)\bi m\b', 'I am', text)
    text = re.sub(r'(?i)\bi d\b', 'I’d', text)
    text = re.sub(r'(?i)\bid\b', 'I’d', text)
    text = re.sub(r'(?i)\bive\b', 'I have', text)
    text = re.sub(r'(?i)\bim ', ' I am ', text)
    text = re.sub(r'(?i)\bi m ', ' I am ', text)
    text = re.sub(r'(?i)\bi d ', ' I’d ', text)
    text = re.sub(r'(?i)\bid ', ' I’d ', text)
    text = re.sub(r'(?i)\bive ', ' I have ', text)
    text = re.sub(r' don t ', ' do not ', text)
    text = re.sub(r' didn t ', ' did not ', text)
    text = re.sub(r' can t ', ' can not ', text)
    text = re.sub(r' aren t ', ' are not ', text)
    text = re.sub(r' couldn t ', ' could not ', text)
    text = re.sub(r' should t ', ' should not ', text)
    text = re.sub(r' haven t ', ' have not ', text)
    text = re.sub(r' won t ', ' will not ', text)
    text = re.sub(r' are t ', ' are not ', text)
    text = re.sub(r' wouldn t ', ' would not ', text)
    text = re.sub(r' was t ', ' was not ', text)
    text = re.sub(r' were t ', ' were not ', text)
    text = re.sub(r' doesn t ', ' does not ', text)
    text = re.sub(r' it s ', ' it is ', text) 
    text = re.sub(r' he s ', ' he is ', text) 
    text = re.sub(r' isn t ', ' is not ', text) 
    text = re.sub(r' there s ', ' there is ', text) 
    text = re.sub(r' that s ', ' that is ', text) 
    text = re.sub(r' u r ', ' you are ', text) 
    # Match una palabra seguida de un espacio + "s" + un espacio
    pattern = r"(\w+)\s+s\s+"
    # Reemplazo: \1's␣ (backreference + apostrofe + s + espacio)
    replacement = r"\1’s "
    text = re.sub(pattern, replacement, text)
    text = re.sub(r' amp ', ' and ', text) 
    text = re.sub(r'\s+', ' ', text).strip()  # eliminar espacios extras
    return text

df['text'] = df['text'].apply(clean_text)


In [62]:
# Eliminar las filas donde emotion sea 'surprise'
df = df[df["emotion"] != "love"].copy()
df = df[df["emotion"] != "surprise"].copy()
df = df.reset_index(drop=True)
# Verificamos que ya no esté 'surprise' en la distribución
print(df["emotion"].value_counts())

emotion
joy        6760
sadness    5797
anger      2709
fear       2373
Name: count, dtype: int64


In [63]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import os

# Ruta base para guardar modelos
base_path = r"C:\Users\leo23\OneDrive\Documentos\Fine_tuning_llama2\models"
model_name = "textattack/roberta-base-CoLA"

# Crear directorio si no existe
model_dir = os.path.join(base_path, "roberta-cola")
os.makedirs(model_dir, exist_ok=True)

# Descargar y guardar modelo solo si no existe
if not os.path.exists(os.path.join(model_dir, "config.json")):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer.save_pretrained(model_dir)
    model.save_pretrained(model_dir)
else:
    print("Modelo ya existe en ruta local, cargando...")

# Cargar desde almacenamiento local
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir).to("cuda" if torch.cuda.is_available() else "cpu")

# Función para evaluar coherencia (igual que antes)
def check_coherence(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits
    prob = torch.softmax(logits, dim=1)
    return prob[0][1].item()

# Aplicar detección de incoherencias (asumiendo que tienes un DataFrame llamado dfe)
df["coherence_score"] = df["text"].apply(check_coherence)
df_incoherent = df[df["coherence_score"] < 0.5]

Modelo ya existe en ruta local, cargando...


In [64]:
df.to_csv("../data/english/english_score_v5.csv", index=False)

In [65]:
# df = pd.read_csv('../data/english/english_score_v1.csv', sep=',')

In [66]:
df

,text,emotion,coherence_score
0,I am feeling rather rotten so I am not very am...,sadness,0.979730
1,I am updating my blog because i feel shitty,sadness,0.972583
2,i never make her separate from me because i do...,sadness,0.689895
3,i left with my bouquet of red and yellow tulip...,joy,0.969233
4,i was feeling a little vain when i did this one,sadness,0.965363
...,...,...,...
17634,I am having ssa examination tomorrow in the mo...,sadness,0.718779
17635,i constantly worry about their fight against n...,joy,0.913532
17636,i feel its important to share this info for th...,joy,0.972394
17637,i truly feel that if you are passionate enough...,joy,0.972112


In [67]:
df_incoherent

,text,emotion,coherence_score
6,i felt anger when at the end of a telephone call,anger,0.167979
12,i pay attention it deepens into a feeling of b...,fear,0.325971
13,i just feel extremely comfortable with the gro...,joy,0.388314
17,i feel just bcoz a fight we get mad to each ot...,anger,0.184463
21,i survey my own posts over the last few years ...,joy,0.496867
...,...,...,...
17621,i wrote feel there rather than think or believ...,joy,0.118154
17623,i feel like the world is just being bitter and...,anger,0.099140
17625,i have i feel pathetic for lying if i say no,sadness,0.187238
17633,i feel a bit rude leaving you hanging there fr...,anger,0.320585


In [68]:
# actualizar dataframe
df = df[df["coherence_score"] > 0.38]

In [69]:
# import pandas as pd
# import dtale
# # Mostrar el DataFrame en D-Tale
# d = dtale.show(df)
# d.open_browser() 

In [70]:
from huggingface_hub import hf_hub_download
import fasttext
import pandas as pd
from tqdm.notebook import tqdm

# Ruta al modelo (usa la ruta completa de tu sistema)
model_path = r"C:\Users\leo23\OneDrive\Documentos\Fine_tuning_llama2\models\fasttext\models--facebook--fasttext-language-identification\snapshots\3af127d4124fc58b75666f3594bb5143b9757e78\model.bin"

# Cargar el modelo
language_detector = fasttext.load_model(model_path)


# 2. Función para detectar idioma con manejo de errores
def detect_language(text: str) -> str:
    try:
        if pd.isna(text) or len(text.strip()) < 3:  # Manejar textos vacíos/cortos
            return "unknown"
        predictions = language_detector.predict(text.replace("\n", " "), k=1)
        return predictions[0][0].split("__")[-1]
    except:
        return "error"

# 3. Aplicar detección a todo el DataFrame con barra de progreso
tqdm.pandas(desc="Detectando idiomas")
df["idioma"] = df["text"].progress_apply(detect_language)

Detectando idiomas:   0%|          | 0/14764 [00:00<?, ?it/s]

C:\Users\leo23\AppData\Local\Temp\ipykernel_16584\3885996047.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
df

,text,emotion,coherence_score,idioma
0,I am feeling rather rotten so I am not very am...,sadness,0.979730,eng_Latn
1,I am updating my blog because i feel shitty,sadness,0.972583,eng_Latn
2,i never make her separate from me because i do...,sadness,0.689895,eng_Latn
3,i left with my bouquet of red and yellow tulip...,joy,0.969233,eng_Latn
4,i was feeling a little vain when i did this one,sadness,0.965363,eng_Latn
...,...,...,...,...
17634,I am having ssa examination tomorrow in the mo...,sadness,0.718779,eng_Latn
17635,i constantly worry about their fight against n...,joy,0.913532,eng_Latn
17636,i feel its important to share this info for th...,joy,0.972394,eng_Latn
17637,i truly feel that if you are passionate enough...,joy,0.972112,eng_Latn


In [72]:
# 4. Analizar resultados
print("Conteo de idiomas identificados:")
df["idioma"].value_counts()

Conteo de idiomas identificados:


idioma
eng_Latn    14730
cat_Latn        8
nld_Latn        7
swe_Latn        5
nno_Latn        2
nob_Latn        2
ita_Latn        2
plt_Latn        1
dan_Latn        1
pol_Latn        1
afr_Latn        1
glg_Latn        1
nso_Latn        1
sot_Latn        1
fuv_Latn        1
Name: count, dtype: int64

In [73]:
df.to_csv("../data/english/english_score_language_v2.csv", index=False)

In [74]:
# import pandas as pd
# import dtale
# # Mostrar el DataFrame en D-Tale
# d = dtale.show(df)
# d.open_browser() 

In [75]:
import pandas as pd
import fasttext
import re

# Filtrar solo textos en español
df = df[df["idioma"] == "eng_Latn"].copy()
df = df.drop(columns=["idioma"])
df = df.drop(columns=["coherence_score"])
df = df.reset_index(drop=True)
df['emotion'].value_counts()
df

,text,emotion
0,I am feeling rather rotten so I am not very am...,sadness
1,I am updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
14725,I am having ssa examination tomorrow in the mo...,sadness
14726,i constantly worry about their fight against n...,joy
14727,i feel its important to share this info for th...,joy
14728,i truly feel that if you are passionate enough...,joy


In [76]:
dataToken = df.copy()

# Función alternativa para contar tokens utilizando split()
def contar_tokens_simple(texto):
    return len(texto.split())

# Aplicar la función y filtrar las filas con menos de 10 tokens
dataToken['token_count'] = df['text'].apply(contar_tokens_simple)
# dataToken = dataToken[dataToken['token_count'] <= 3]

# Mostrar el resultado
dataToken

,text,emotion,token_count
0,I am feeling rather rotten so I am not very am...,sadness,13
1,I am updating my blog because i feel shitty,sadness,9
2,i never make her separate from me because i do...,sadness,22
3,i left with my bouquet of red and yellow tulip...,joy,21
4,i was feeling a little vain when i did this one,sadness,11
...,...,...,...
14725,I am having ssa examination tomorrow in the mo...,sadness,36
14726,i constantly worry about their fight against n...,joy,29
14727,i feel its important to share this info for th...,joy,15
14728,i truly feel that if you are passionate enough...,joy,19


In [77]:
# import pandas as pd
# import dtale

# # Mostrar el DataFrame en D-Tale
# d = dtale.show(dataToken)
# d.open_browser() 

In [78]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Verificar si hay GPU disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [79]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# # Directorio donde se guardarán los archivos
# model_dir = "../models/opus-mt-en-es"

# # Descargar y guardar el modelo y tokenizador
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es", cache_dir=model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es", cache_dir=model_dir)

# # Guardar explícitamente en el directorio (opcional)
# tokenizer.save_pretrained(model_dir)
# model.save_pretrained(model_dir)

In [80]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Ruta absoluta al modelo local
model_dir = r"C:\Users\leo23\OneDrive\Documentos\Fine_tuning_llama2\models\opus-mt-en-es"

# Cargar tokenizador y modelo
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda")

# Probar traducción
text = "Hello, world!"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs)
translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated)  # Debería imprimir: "¡Hola, mundo!"

C:\Users\leo23\OneDrive\Documentos\Fine_tuning_llama2\venv\Lib\site-packages\transformers\generation\configuration_utils.py:526: UserWarning:

`num_beams` is set to None - defaulting to 1.



¡Hola, mundo!


In [81]:
# # Función para traducir con batch processing
# def translate_texts(texts, batch_size=16):
#     translated_texts = []
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i+batch_size]
#         inputs = tokenizer(batch.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
#         outputs = model.generate(**inputs)
#         translated_batch = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
#         translated_texts.extend(translated_batch)
#     return translated_texts

# # Traducir dataset por lotes
# df["text"] = translate_texts(df["text"])



In [82]:
# --------------------------------------------------
# Función de traducción optimizada
# --------------------------------------------------
import torch
from tqdm.notebook import tqdm  # Para mostrar progreso
def translate_texts(texts, batch_size=8, max_length=512):
    """
    Traduce un lote de textos inglés->español
    :param texts: Lista/Serie de textos a traducir
    :param batch_size: Tamaño del lote
    :param max_length: Longitud máxima de tokens
    """
    translated_texts = []
    # Configurar modelo para inferencia
    model.eval()
    # Procesar por lotes con barra de progreso
    for i in tqdm(range(0, len(texts), batch_size), desc="Traduciendo"):
        try:
            batch = texts[i:i+batch_size].tolist()
            
            # Tokenizar lote
            inputs = tokenizer(
                batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=max_length,
                pad_to_multiple_of=8 if device == "cuda" else None  # Optimización para GPU
            ).to(device)
            
            # Generar traducciones
            with torch.no_grad():
                outputs = model.generate(**inputs)
            
            # Decodificar
            translated_batch = tokenizer.batch_decode(
                outputs,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            
            translated_texts.extend(translated_batch)
            
        except Exception as e:
            print(f"Error en lote {i//batch_size}: {str(e)}")
            translated_texts.extend([""] * len(batch))  # Añadir placeholders
    
    return translated_texts

# --------------------------------------------------
# Traducir todo el dataset
# --------------------------------------------------
print("\nIniciando traducción completa...")
df["text"] = translate_texts(df["text"], batch_size=16 if device == "cuda" else 4)




Iniciando traducción completa...


Traduciendo:   0%|          | 0/921 [00:00<?, ?it/s]

In [83]:
# --------------------------------------------------
# 5. Guardar resultados
# --------------------------------------------------
# Guarda el resultado en un nuevo archivo CSV
df.to_csv("../data/english/dataset_traducido4.csv", index=False)
print(f"\nTraducción completada!")

print(df.head(3))


Traducción completada!
                                                text  emotion
0  Me siento un poco podrido, así que no soy muy ...  sadness
1  Estoy actualizando mi blog porque me siento un...  sadness
2  Nunca la hago separar de mí porque nunca quier...  sadness


In [84]:
df

,text,emotion
0,"Me siento un poco podrido, así que no soy muy ...",sadness
1,Estoy actualizando mi blog porque me siento un...,sadness
2,Nunca la hago separar de mí porque nunca quier...,sadness
3,Me fui con mi ramo de tulipanes rojos y amaril...,joy
4,Me sentía un poco vanidosa cuando hice esto.,sadness
...,...,...
14725,Estoy haciendo el examen de ssa mañana por la ...,sadness
14726,Me preocupo constantemente por su lucha contra...,joy
14727,Siento que es importante compartir esta inform...,joy
14728,Realmente siento que si eres lo suficientement...,joy


In [85]:
# Mapeo de etiquetas de inglés a español
emotion_mapping = {
    'anger': 'ira',
    'disgust': 'disgusto',
    'sadness': 'tristeza',
    'joy': 'alegría',
    'fear': 'miedo',
}

df['emotion'] = df['emotion'].map(emotion_mapping) # Nueva columna 'emotion' con valores en español

In [86]:
import pandas as pd

# Eliminar todos los puntos al final de cada string
df["text"] = df["text"].str.rstrip(".")

# Eliminar los puntos suspensivos de la columna 'text'
# df['text'] = df['text'].str.replace('.', '', regex=False)



In [87]:
# df.to_csv("../data/english/translated_cleaned2.csv", index=False)

In [88]:
# df = pd.read_csv('../data/english/translated_cleaned2.csv', sep=',')

In [89]:
dataToken = df.copy()

# Función alternativa para contar tokens utilizando split()
def contar_tokens_simple(texto):
    return len(texto.split())

# Aplicar la función y filtrar las filas con menos de 10 tokens
dataToken['token_count'] = dataToken['text'].apply(contar_tokens_simple)


In [90]:
df_menos_de_3_tokens = dataToken[dataToken['token_count'] < 3]

# Mostrar el resultado
df_menos_de_3_tokens

,text,emotion,token_count
10446,una noche,alegría,2
10887,un día,tristeza,2
11191,Tengo miedo,miedo,2
11280,sin descripción,ira,2
14469,una tarde,miedo,2


In [91]:
# Obtener los índices de las filas donde 'token_count' ES MENOR A 3
indices_a_eliminar = dataToken[dataToken['token_count'] < 3].index

# Eliminar las filas del DataFrame original (df)
df = df.drop(index=indices_a_eliminar)
dataToken = dataToken.drop(index=indices_a_eliminar)
# Reiniciar los índices de ambos DataFrames
df = df.reset_index(drop=True)
dataToken = dataToken.reset_index(drop=True)

In [92]:
df_mas_de_60_tokens = dataToken[dataToken['token_count'] > 61]
df_mas_de_60_tokens

,text,emotion,token_count
465,No pude evitar sentir por él y esta terrible s...,tristeza,63
3526,Necesitaba un plan sobre cómo deshacerme de es...,ira,62
4866,Sé que no todas las mujeres se sienten de esta...,tristeza,63
6432,"Siento algo dentro de Paul diciendo: ""A la mie...",tristeza,63
12043,Tengo una especie de aumento de la frecuencia ...,ira,64
12062,Me sentía muy aislada en el extremo norte de E...,tristeza,62
12723,Tengo que empujar hacia atrás las expresiones ...,ira,65


In [93]:
df.to_csv("../data/english/translated_cleaned4.csv", index=False)

In [40]:
import pandas as pd
import dtale

# Mostrar el DataFrame en D-Tale
d = dtale.show(df)
d.open_browser() 

In [94]:
# Lista de letras a eliminar
letras = ['b', 'c', 'd', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'ñ', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']

# Compilar patrón: \b(l1|l2|...)\b
pattern = re.compile(r"\b(" + "|".join(letras) + r")\b", flags=re.IGNORECASE)

# Remover letras sueltas y limpiar espacios extra
df['text'] = df['text'].apply(lambda x: pattern.sub("", x))
df['text'] = df['text'].str.replace(r"\s+", " ", regex=True).str.strip()


In [95]:
df.to_csv("../data/english/translated_cleaned5.csv", index=False)

2025-05-18 22:00:54,514 - INFO     - Executing shutdown due to inactivity...
2025-05-18 22:00:58,837 - INFO     - Executing shutdown...
2025-05-18 22:00:58,839 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
C:\Users\leo23\OneDrive\Documentos\Fine_tuning_llama2\venv\Lib\site-packages\dtale\app.py:445: FutureWarning:

`torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead

2025-05-18 22:00:59,758 - ERROR    - weakly-referenced object no longer exists
2025-05-18 22:00:59,760 - ERROR    - weakly-referenced object no longer exists


In [41]:
# df = df[df["emotion"] == "miedo"].copy()

# # Verificar la distribución
# print(df["emotion"].value_counts())